# クーラビ part

In [ ]:
import sys
sys.path.append('../input/timm-3monthsold/pytorch-image-models-master 2')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
import os, gc
import random
from PIL import Image
import tifffile as tiff
import cv2
import zipfile
import collections
from PIL import Image
from sklearn import preprocessing
from random import randint
from glob import glob
import shutil
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss
import torchvision
from torchvision import transforms
import albumentations as A
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim.lr_scheduler import CosineAnnealingLR
import warnings
warnings.filterwarnings("ignore")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed = 2020
seed_everything(seed)
sz1 = 224
sz2 = 384
NFOLDS = 5
pet_num = 4

#ImageNet
mean = np.array([[[0.485, 0.456, 0.406]]])
std = np.array([[[0.229, 0.224, 0.225]]])

In [ ]:
test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
#test_df = pd.concat([pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')]*900, ignore_index=True)
test_df['class'] = -1
test_df['conf'] = -1
test_ids = test_df.Id.to_list()
#test_ids = ['0013fd999caf9a3efe1352ca1b0d937e', '0009c66b9439883ba2750fb825e1d7db', '0007de18844b0dbbb5e1f607da0606e0']
test_dir = "/kaggle/input/petfinder-pawpularity-score/test/"
#test_dir = '/kaggle/input/petfinder2-sample-images/'
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')

In [ ]:
tmp = pd.DataFrame(columns=test_df.columns)
tmp

In [ ]:
!python detect.py\
--weights /kaggle/input/ultralyticsyolov5aweights/yolov5x.pt\
--class 15 16\
--img 512\
--conf 0.3\
--iou 0.5\
--source $test_dir\
--name inference\
--save-txt --save-conf --exist-ok

In [ ]:
os.chdir('/kaggle/working')
save_dir = f'/kaggle/working/crop_images/'
os.makedirs(save_dir, exist_ok=True)

In [ ]:
for n, image_id in tqdm(enumerate(test_ids)):
    orig_image = cv2.imread(f'/kaggle/input/petfinder-pawpularity-score/test/{image_id}.jpg')
    orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    height = orig_image.shape[0]
    width = orig_image.shape[1]
    try:
        file_path = f'/kaggle/working/yolov5/runs/detect/inference/labels/{image_id}.txt'
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        data = data[np.argsort(data[:, 1])[::-1]]  #sort by conf
        for i, d in enumerate(data):
            xmin = int((d[2]-d[4]/2)*width)
            ymin = int((d[3]-d[5]/2)*height)
            xmax = int((d[2]+d[4]/2)*width)
            ymax = int((d[3]+d[5]/2)*height)
            width_half = (xmax - xmin) // 2
            height_half = (ymax - ymin) // 2
            r = np.maximum(width_half, height_half)
            xc = (xmin + xmax) // 2
            yc = (ymin + ymax) // 2
            final_xmin = np.maximum(xc-r, 0)
            final_ymin = np.maximum(yc-r, 0)
            final_xmax = np.minimum(xc+r, width)
            final_ymax = np.minimum(yc+r, height)
            crop_img = orig_image[final_ymin:final_ymax, final_xmin:final_xmax, :]
            np.save(save_dir + f'{image_id}-{i}', crop_img.astype(np.uint8))
            df = pd.DataFrame(columns=test_df.columns)
            df.loc[0, 'Id'] = f'{image_id}-{i}'
            df.loc[0, 'class'] = d[0]
            df.loc[0, 'conf'] = d[1]
            tmp = tmp.append(df, ignore_index=True)
            
    except:
        np.save(save_dir + f'{image_id}-0', orig_image.astype(np.uint8))
        df = pd.DataFrame(columns=test_df.columns)
        df.loc[0, 'Id'] = f'{image_id}-0'
        df.loc[0, 'class'] = 'NA'
        df.loc[0, 'conf'] = 'NA'
        tmp = tmp.append(df, ignore_index=True)

In [ ]:
tmp = tmp.rename(columns={'Id': 'Id2'})
tmp['img_idx'] = tmp['Id2'].apply(lambda x: x.split('-')[1])
tmp['Id'] = tmp['Id2'].apply(lambda x: x.split('-')[0])
tmp

In [ ]:
class Dataset(Dataset):
    def __init__(self, df, size, transform=None):
        self.df = df
        self.size = size
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_id = self.df.loc[idx, 'Id2']
        img = np.load(f'/kaggle/working/crop_images/{image_id}.npy')
        img = cv2.resize(img, (self.size, self.size)).astype(np.float32)
        
        if self.transform:
            sample = self.transform(image=img)
            img = sample['image']
        
        img = (img/255.0 - mean) / std
        img = np.transpose(img, (2, 0, 1))
        img = torch.from_numpy(img)

        return img
    
    
class Dataset2(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_id = self.df.loc[idx, 'Id']
        img = np.load(f'/kaggle/working/crop_images/{image_id}.npy').astype(np.float32)
        
        if self.transform:
            sample = self.transform(image=img)
            img = sample['image']
        
        img = (img/255.0 - mean) / std
        img = np.transpose(img, (2, 0, 1))
        img = torch.from_numpy(img)

        return img
    

def inference_fn1(data_loader, model, device):
    model.eval()    
    val_preds = []
    
    for i, x in enumerate(data_loader):
        img = x
        img = img.to(device, dtype=torch.float)
        
        with torch.no_grad():
            pred = model(img)
            val_preds.append(nn.Softmax()(pred).detach().cpu().numpy())
            
    val_preds = np.concatenate(val_preds)
                
    return val_preds


def inference_fn2(data_loader, model, device):
    model.eval()    
    val_preds = []
    
    for i, x in enumerate(data_loader):
        img = x
        img = img.to(device, dtype=torch.float)
        
        with torch.no_grad():
            pred = model(img)
            val_preds.append(nn.Sigmoid()(pred).detach().cpu().numpy() * 100)
            
    val_preds = np.concatenate(val_preds)
                
    return val_preds


class Model(nn.Module):
    def __init__(self, model_name=None, pretrained=False, num_classes=100):
        super().__init__()
        self.model = timm.create_model(
            model_name=model_name, 
            in_chans=3, 
            pretrained=pretrained
            )
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
#224x224
#5folds
#cropped imgs
test_ds = Dataset(df=tmp, size=sz1, transform=None)
test_dl = DataLoader(dataset=test_ds, batch_size=256, shuffle=False, num_workers=2)
target_cols = np.arange(1, 101)
predictions1 = 0
predictions2 = 0

for fold in range(NFOLDS):
    print(f'======== FOLD:{fold} inference ========')
    
    #SwinT type1: 100 outputs
    model1 = Model(model_name='swin_large_patch4_window7_224_in22k', num_classes=100)
    model1.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint1-weight-016/swint1_fold_{fold}.pth'))
    model1.to(device)
    prediction1 = inference_fn1(test_dl, model1, device)
    predictions1 += (target_cols.reshape(-1, 100) * prediction1.reshape(-1, 100)).sum(axis=1) / NFOLDS
    
    #SwinT type2: 1 output
    model2 = Model(model_name='swin_large_patch4_window7_224_in22k', num_classes=1)
    model2.model.head = nn.Sequential(
        nn.Linear(model2.model.head.in_features, 128),
        nn.Dropout(0.0),
        nn.Linear(128, 64),
        nn.Linear(64, 1)
    )
    model2.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint2-weight-011/swint2_fold_{fold}.pth'))
    model2.to(device)
    prediction2 = inference_fn2(test_dl, model2, device).flatten()
    predictions2 += prediction2 / NFOLDS
    
    del model1, model2, prediction1, prediction2
    gc.collect()
    
tmp['pred1'] = predictions1
tmp['pred2'] = predictions2

del test_ds, test_dl, predictions1, predictions2
gc.collect()

In [ ]:
#384x384
#5folds
#cropped imgs
test_ds = Dataset(df=tmp, size=sz2, transform=None)
test_dl = DataLoader(dataset=test_ds, batch_size=64, shuffle=False, num_workers=2)
target_cols = np.arange(1, 101)
predictions3 = 0
predictions4 = 0

for fold in range(NFOLDS):
    print(f'======== FOLD:{fold} inference ========')
    
    #SwinT type1: 100 outputs
    model1 = Model(model_name='swin_large_patch4_window12_384_in22k', num_classes=100)
    model1.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint1-weight-068/swint1_fold_{fold}.pth'))
    model1.to(device)
    prediction3 = inference_fn1(test_dl, model1, device)
    predictions3 += (target_cols.reshape(-1, 100) * prediction3.reshape(-1, 100)).sum(axis=1) / NFOLDS
    
    #SwinT type2: 1 output
    model2 = Model(model_name='swin_large_patch4_window12_384_in22k', num_classes=1)
    model2.model.head = nn.Sequential(
        nn.Linear(model2.model.head.in_features, 128),
        nn.Dropout(0.0),
        nn.Linear(128, 64),
        nn.Linear(64, 1)
    )
    model2.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint2-weight-028/swint2_fold_{fold}.pth'))
    model2.to(device)
    prediction4 = inference_fn2(test_dl, model2, device).flatten()
    predictions4 += prediction4 / NFOLDS
    
    del model1, model2, prediction3, prediction4
    gc.collect()
    
tmp['pred3'] = predictions3
tmp['pred4'] = predictions4

del test_ds, test_dl, predictions3, predictions4
gc.collect()

In [ ]:
tmp

In [ ]:
#postprocessing by multi-pets
test_df = tmp.groupby('Id').head(pet_num).groupby('Id').mean().reset_index()
test_df

In [ ]:
sub = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
coorabi = pd.concat(
    [sub, 
     test_df[['pred1']], 
     test_df[['pred2']], 
     test_df[['pred3']], 
     test_df[['pred4']]
    ], axis=1
)
coorabi

In [ ]:
!rm -r /kaggle/working/yolov5
!rm -r /kaggle/working/crop_images

In [ ]:
#fastai
#224x224
#5folds
#original imgs
import fastai
from fastai.vision.all import *
from fastai.callback.all import *
import torchvision.models as torch_models

def petfinder_rmse(input, target):
    return 100 * torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
test_df['Path'] = '../input/petfinder-pawpularity-score/test/' + test_df['Id'] + '.jpg'
predictions5 = 0

for fold in range(NFOLDS):
    print(f'======== FOLD:{fold} inference ========')
    learn = load_learner(fname = Path(f'/kaggle/input/petfinder2-swint3-weight-001/swint3_fold_{fold}.pkl'), cpu=False)
    test_dl = learn.dls.test_dl(test_df)
    preds, _ = learn.get_preds(dl=test_dl)
    predictions5 += preds * 100 / NFOLDS
    del learn, test_dl, preds
    gc.collect()
    
coorabi['pred5'] = predictions5

del predictions5
gc.collect()

In [ ]:
#224x224
#5folds
#center crop
test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
test_ids = test_df.Id.to_list()
os.chdir('/kaggle/working')
save_dir = f'/kaggle/working/crop_images/'
os.makedirs(save_dir, exist_ok=True)

for n, image_id in tqdm(enumerate(test_ids)):
    orig_image = cv2.imread(f'/kaggle/input/petfinder-pawpularity-score/test/{image_id}.jpg')
    orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    height = orig_image.shape[0]
    width = orig_image.shape[1]
    xc = width // 2
    yc = height // 2
    r = np.minimum(xc, yc)
    xmin = np.maximum(xc-r, 0)
    ymin = np.maximum(yc-r, 0)
    xmax = np.minimum(xc+r, width)
    ymax = np.minimum(yc+r, height)
    crop_img = orig_image[ymin:ymax, xmin:xmax, :]
    crop_img = cv2.resize(crop_img, (sz1, sz1)).astype(np.uint8)
    np.save(save_dir + f'{image_id}', crop_img)

test_ds = Dataset2(df=coorabi, transform=None)
test_dl = DataLoader(dataset=test_ds, batch_size=256, shuffle=False, num_workers=2)
target_cols = np.arange(1, 101)
predictions6 = 0
predictions7 = 0

for fold in range(NFOLDS):
    print(f'======== FOLD:{fold} inference ========')
    
    #SwinT type1: 100 outputs
    model1 = Model(model_name='swin_large_patch4_window7_224_in22k', num_classes=100)
    model1.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint1-weight-076/swint1_fold_{fold}.pth'))
    model1.to(device)
    prediction6 = inference_fn1(test_dl, model1, device)
    predictions6 += (target_cols.reshape(-1, 100) * prediction6.reshape(-1, 100)).sum(axis=1) / NFOLDS
    
    #SwinT type2: 1 output
    model2 = Model(model_name='swin_large_patch4_window7_224_in22k', num_classes=1)
    model2.model.head = nn.Sequential(
        nn.Linear(model2.model.head.in_features, 128),
        nn.Dropout(0.0),
        nn.Linear(128, 64),
        nn.Linear(64, 1)
    )
    model2.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint2-weight-036/swint2_fold_{fold}.pth'))
    model2.to(device)
    prediction7 = inference_fn2(test_dl, model2, device).flatten()
    predictions7 += prediction7 / NFOLDS
    
    del model1, model2, prediction6, prediction7
    gc.collect()
    
coorabi['pred6'] = predictions6
coorabi['pred7'] = predictions7

!rm -r /kaggle/working/crop_images
del test_ds, test_dl, predictions6, predictions7
gc.collect()

In [ ]:
#384x384
#5folds
#center crop
test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
test_ids = test_df.Id.to_list()
os.chdir('/kaggle/working')
save_dir = f'/kaggle/working/crop_images/'
os.makedirs(save_dir, exist_ok=True)

for n, image_id in tqdm(enumerate(test_ids)):
    orig_image = cv2.imread(f'/kaggle/input/petfinder-pawpularity-score/test/{image_id}.jpg')
    orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    height = orig_image.shape[0]
    width = orig_image.shape[1]
    xc = width // 2
    yc = height // 2
    r = np.minimum(xc, yc)
    xmin = np.maximum(xc-r, 0)
    ymin = np.maximum(yc-r, 0)
    xmax = np.minimum(xc+r, width)
    ymax = np.minimum(yc+r, height)
    crop_img = orig_image[ymin:ymax, xmin:xmax, :]
    crop_img = cv2.resize(crop_img, (sz2, sz2)).astype(np.uint8)
    np.save(save_dir + f'{image_id}', crop_img)

test_ds = Dataset2(df=coorabi, transform=None)
test_dl = DataLoader(dataset=test_ds, batch_size=64, shuffle=False, num_workers=2)
target_cols = np.arange(1, 101)
predictions8 = 0
predictions9 = 0

for fold in range(NFOLDS):
    print(f'======== FOLD:{fold} inference ========')
    
    #SwinT type1: 100 outputs
    model1 = Model(model_name='swin_large_patch4_window12_384_in22k', num_classes=100)
    model1.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint1-weight-077/swint1_fold_{fold}.pth'))
    model1.to(device)
    prediction8 = inference_fn1(test_dl, model1, device)
    predictions8 += (target_cols.reshape(-1, 100) * prediction8.reshape(-1, 100)).sum(axis=1) / NFOLDS
    
    #SwinT type2: 1 output
    model2 = Model(model_name='swin_large_patch4_window12_384_in22k', num_classes=1)
    model2.model.head = nn.Sequential(
        nn.Linear(model2.model.head.in_features, 128),
        nn.Dropout(0.0),
        nn.Linear(128, 64),
        nn.Linear(64, 1)
    )
    model2.load_state_dict(torch.load(f'/kaggle/input/petfinder2-swint2-weight-037/swint2_fold_{fold}.pth'))
    model2.to(device)
    prediction9 = inference_fn2(test_dl, model2, device).flatten()
    predictions9 += prediction9 / NFOLDS
    
    del model1, model2, prediction8, prediction9
    gc.collect()
    
coorabi['pred8'] = predictions8
coorabi['pred9'] = predictions9

!rm -r /kaggle/working/crop_images
del test_ds, test_dl, predictions8, predictions9
gc.collect()

In [ ]:
coorabi['Pawpularity'] = ((coorabi['pred1'] * 0.275 + coorabi['pred2'] * 0.225 + coorabi['pred3'] * 0.275 + coorabi['pred4'] * 0.225) * 0.75 + coorabi['pred5'] * 0.25) * 0.6 + (coorabi['pred6'] * 0.22 + coorabi['pred7'] * 0.18 + coorabi['pred8'] * 0.33 + coorabi['pred9'] * 0.27) * 0.4

In [ ]:
coorabi

In [ ]:
coorabi[['Id', 'Pawpularity']].to_csv('/kaggle/working/submission.csv', index=False)